In [1]:
# # Instalando blibiotecas
# !pip install pandas
# !pip install scikit-learn
# !pip install numpy
# !pip install xgboost
# !pip install lightgbm
# !pip install catboost

In [1]:
# ===== SESSÃO 1 + 2 + 3 (VERSÃO OTIMIZADA PARA GRANDES VOLUMES) =====
import pandas as pd
import os

print("=== PROCESSANDO DADOS DE VOOS (modo otimizado) ===")
base_folder_path = './dataset/'

# Arquivo temporário (streaming)
TEMP_FILE = "temp_voos.csv"
if os.path.exists(TEMP_FILE):
    os.remove(TEMP_FILE)

df_list = []
first_write = True

all_items = os.listdir(base_folder_path)
year_folders = [d for d in all_items if os.path.isdir(os.path.join(base_folder_path, d))]

# ===== PARTE 1 — LEITURA EM CHUNKS (SEM ESTOURAR MEMÓRIA) =====
for year_folder in year_folders:
    year_path = os.path.join(base_folder_path, year_folder)

    try:
        year_from_folder = int(year_folder)
    except ValueError:
        print(f"Ignorando pasta '{year_folder}' pois o nome não é ano.")
        continue

    year_files = os.listdir(year_path)
    # Ignorar arquivos .~lock
    csv_files = [
        f for f in year_files
        if f.endswith('.CSV') and not f.startswith('.~lock')
    ]

    for csv_file in csv_files:
        file_path = os.path.join(year_path, csv_file)
        print(f"Lendo (chunks) → {file_path}")

        try:
            chunk_iter = pd.read_csv(file_path, sep=';', chunksize=100_000)
        except Exception as e:
            print(f"Erro ao abrir {csv_file}: {e}")
            continue

        for chunk in chunk_iter:
            # TRATAMENTO DO CHUNK
            chunk['ANO'] = year_from_folder
            chunk['MES'] = chunk['MES'].astype(str).str.replace(',', '', regex=False)
            chunk['MES'] = pd.to_numeric(chunk['MES'], errors='coerce').fillna(0).astype(int)

            chunk['DATA'] = pd.to_datetime(
                chunk['ANO'].astype(str) + '-' + chunk['MES'].astype(str),
                format='%Y-%m', errors='coerce'
            )
            chunk['ANO'] = chunk['DATA'].dt.year
            chunk['MES'] = chunk['DATA'].dt.month

            chunk = chunk.drop(columns=['nr_ano_referencia', 'nr_mes_referencia', 'DATA'], errors='ignore')

            # Salvar em streaming no arquivo temporário
            chunk.to_csv(TEMP_FILE, mode='a', index=False, header=first_write)
            first_write = False

print("Leitura dos voos concluída. Criando main_df...")

# Carregar arquivo já unido
try:
    main_df = pd.read_csv(TEMP_FILE)
    print(f"main_df criado com sucesso! Shape: {main_df.shape}")
except Exception as e:
    print("Erro ao criar main_df:", e)
    main_df = pd.DataFrame()

# Reordenar colunas
if not main_df.empty:
    cols = main_df.columns.tolist()
    if 'MES' in cols and 'ANO' in cols and 'ASSENTOS' in cols:
        cols.remove('MES')
        cols.remove('ANO')
        cols.insert(cols.index('ASSENTOS') + 1, 'MES')
        cols.insert(cols.index('ASSENTOS') + 2, 'ANO')
        main_df = main_df[cols]

    # Mapear empresas
    empresa_mapping = {
        'ABJ': 'ATA - AEROTÁXI ABAETÉ LTDA.',
        'AZU': 'AZUL LINHAS AÉREAS BRASILEIRAS S/A',
        'GLO': 'GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS S.A.)',
        'TAM': 'TAM LINHAS AÉREAS S.A.',
        'CQB': 'APUÍ TÁXI AÉREO S/A'
    }
    if 'EMPRESA' in main_df.columns:
        main_df['EMPRESA'] = main_df['EMPRESA'].replace(empresa_mapping)
        print("Mapeamento de empresas concluído!")

# ===== PARTE 2 — AEROPORTOS =====
print("\n=== PROCESSANDO DADOS DE AEROPORTOS ===")
csv_file_path = './dataset/cadastro-de-aerodromos-civis-publicos.csv'

try:
    airport_df = pd.read_csv(csv_file_path, sep=';')
    airport_df = airport_df[['CÓDIGO OACI', 'MUNICÍPIO ATENDIDO', 'UF']]
    print("airport_df criado!", airport_df.shape)
except Exception as e:
    print("Erro ao carregar airport_df:", e)
    airport_df = pd.DataFrame()

# ===== PARTE 3 — MERGE FINAL EM CHUNKS (SEM ESTOURAR A MEMÓRIA) =====

print("\n=== INICIANDO MERGE FINAL EM CHUNKS ===")

MERGED_FILE = "temp_merged.csv"
if os.path.exists(MERGED_FILE):
    os.remove(MERGED_FILE)

if main_df.empty or airport_df.empty:
    print("ERRO: Um dos DataFrames está vazio.")
else:
    # Padronizar airport_df apenas 1 vez
    airport_df_clean = airport_df.copy()
    airport_df_clean['CÓDIGO OACI'] = airport_df_clean['CÓDIGO OACI'].astype(str).str.strip().str.upper()

    # Ler main_df em chunks direto do arquivo temp_voos.csv
    CHUNK_SIZE = 200_000

    first_write_merge = True

    for chunk in pd.read_csv(TEMP_FILE, chunksize=CHUNK_SIZE):

        print(f"Processando merge chunk → {len(chunk)} linhas...")

        # Padronizar
        chunk['ORIGEM'] = chunk['ORIGEM'].astype(str).str.strip().str.upper()
        chunk['DESTINO'] = chunk['DESTINO'].astype(str).str.strip().str.upper()

        # 🔹 MERGE ORIGEM
        chunk = chunk.merge(
            airport_df_clean,
            left_on='ORIGEM',
            right_on='CÓDIGO OACI',
            how='left'
        ).rename(columns={'MUNICÍPIO ATENDIDO': 'MUNICIPIO_ORIGEM'}) \
         .drop(columns=['CÓDIGO OACI', 'UF'], errors='ignore')

        # 🔹 MERGE DESTINO
        chunk = chunk.merge(
            airport_df_clean,
            left_on='DESTINO',
            right_on='CÓDIGO OACI',
            how='left'
        ).rename(columns={'MUNICÍPIO ATENDIDO': 'MUNICIPIO_DESTINO'}) \
         .drop(columns=['CÓDIGO OACI', 'UF'], errors='ignore')

        # Reorganizar colunas
        if 'MUNICIPIO_ORIGEM' in chunk.columns:
            chunk = chunk.rename(columns={'MUNICIPIO_ORIGEM': 'ORIGEM'})
        if 'MUNICIPIO_DESTINO' in chunk.columns:
            chunk = chunk.rename(columns={'MUNICIPIO_DESTINO': 'DESTINO'})

        # Remover linhas inválidas
        chunk = chunk.dropna(subset=['ORIGEM', 'DESTINO'])

        # 🔥 Salvar incrementalmente
        chunk.to_csv(MERGED_FILE, index=False, mode='a', header=first_write_merge)
        first_write_merge = False

    print("=== MERGE EM CHUNKS CONCLUÍDO ===")

# Carregar DataFrame final consolidado
final_df = pd.read_csv(MERGED_FILE)

print("\n=== RESULTADO FINAL ===")
print(f"Shape final: {final_df.shape}")
print(final_df.head())

=== PROCESSANDO DADOS DE VOOS (modo otimizado) ===
Lendo (chunks) → ./dataset/2023/202301.CSV
Lendo (chunks) → ./dataset/2023/202304.CSV
Lendo (chunks) → ./dataset/2023/202310.CSV
Lendo (chunks) → ./dataset/2023/202303.CSV
Lendo (chunks) → ./dataset/2023/202308.CSV
Lendo (chunks) → ./dataset/2023/202312.CSV
Lendo (chunks) → ./dataset/2023/202309.CSV
Lendo (chunks) → ./dataset/2023/202305.CSV
Lendo (chunks) → ./dataset/2023/202302.CSV
Lendo (chunks) → ./dataset/2023/202311.CSV
Lendo (chunks) → ./dataset/2023/202306.CSV
Lendo (chunks) → ./dataset/2023/202307.CSV
Lendo (chunks) → ./dataset/2024/202407.CSV
Lendo (chunks) → ./dataset/2024/202401.CSV
Lendo (chunks) → ./dataset/2024/202402.CSV
Lendo (chunks) → ./dataset/2024/202405.CSV
Lendo (chunks) → ./dataset/2024/202411.CSV
Lendo (chunks) → ./dataset/2024/202403.CSV
Lendo (chunks) → ./dataset/2024/202404.CSV
Lendo (chunks) → ./dataset/2024/202412.CSV
Lendo (chunks) → ./dataset/2024/202409.CSV
Lendo (chunks) → ./dataset/2024/202410.CSV
Len

/tmp/ipykernel_9360/2882746123.py:69: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  main_df = pd.read_csv(TEMP_FILE)


main_df criado com sucesso! Shape: (17873941, 7)
Mapeamento de empresas concluído!

=== PROCESSANDO DADOS DE AEROPORTOS ===
airport_df criado! (504, 3)

=== INICIANDO MERGE FINAL EM CHUNKS ===
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...


/tmp/ipykernel_9360/2882746123.py:129: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(TEMP_FILE, chunksize=CHUNK_SIZE):


Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...


...

Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...


/tmp/ipykernel_9360/2882746123.py:129: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(TEMP_FILE, chunksize=CHUNK_SIZE):


Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 200000 linhas...
Processando merge chunk → 73941 linhas...
=== MERGE EM CHUNKS CONCLUÍDO ===


/tmp/ipykernel_9360/2882746123.py:171: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  final_df = pd.read_csv(MERGED_FILE)



=== RESULTADO FINAL ===
Shape final: (17765412, 9)
    ANO  MES EMPRESA ORIGEM DESTINO  TARIFA  ASSENTOS ORIGEM.1       DESTINO.1
0  2023    1     AZU   SBAC    SBAR  851,90         2  ARACATI         ARACAJU
1  2023    1     AZU   SBAC    SBFI  734,90         2  ARACATI   FOZ DO IGUAÇU
2  2023    1     AZU   SBAC    SBGL  832,90         1  ARACATI  RIO DE JANEIRO
3  2023    1     AZU   SBAC    SBGR  630,90         4  ARACATI       GUARULHOS
4  2023    1     AZU   SBAC    SBGR  928,90         2  ARACATI       GUARULHOS


In [2]:
# Preparação dos dados para modelagem com otimização de memória
from sklearn.preprocessing import LabelEncoder
import numpy as np

df = final_df  # renomear para facilitar

# Otimizar tipos de dados para reduzir uso de memória
df['TARIFA'] = df['TARIFA'].astype(str).str.replace(',', '.').astype('float32')
df['MES'] = df['MES'].astype('int8')
df['ANO'] = df['ANO'].astype('int16')
df['ASSENTOS'] = df['ASSENTOS'].astype('int32')

# Verificar cardinalidade das colunas categóricas
print("Número de valores únicos em cada coluna:")
for col in ['EMPRESA', 'ORIGEM', 'DESTINO']:
    print(f"{col}: {df[col].nunique()} valores únicos")

# Variável alvo
y = df['TARIFA']
X = df[['EMPRESA', 'ORIGEM', 'DESTINO', 'MES', 'ANO']].copy()

# Aplicar Label Encoding para colunas categóricas
encoders = {}
for col in ['EMPRESA', 'ORIGEM', 'DESTINO']:
    le = LabelEncoder()
    X[col] = le.fit_transform(df[col].astype(str))
    encoders[col] = le
    X[col] = X[col].astype('int32')  # reduzir memória

print("\nShape do DataFrame X após encoding:", X.shape)
print("\nUso de memória otimizado para as features.")


Número de valores únicos em cada coluna:
EMPRESA: 5 valores únicos
ORIGEM: 170 valores únicos
DESTINO: 169 valores únicos

Shape do DataFrame X após encoding: (17765412, 5)

Uso de memória otimizado para as features.


In [3]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Dividir dados
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

# K-Fold
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return r2, rmse

print("Dados divididos e validação cruzada configurada.")

Dados divididos e validação cruzada configurada.


In [4]:
import lightgbm as lgb
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Parâmetros otimizados
lgb_params = {
    'objective': 'regression',
    'n_estimators': 200,
    'learning_rate': 0.05,
    'num_leaves': 31,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'random_state': 42,
    'n_jobs': -1,
    'early_stopping_rounds': 20,
    'verbose': -1
}

# Treinar LightGBM
lgb_model = lgb.LGBMRegressor(**lgb_params)

lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='rmse'
)

# Avaliar
y_pred_lgbm = lgb_model.predict(X_test)
r2_lgbm = r2_score(y_test, y_pred_lgbm)
rmse_lgbm = np.sqrt(mean_squared_error(y_test, y_pred_lgbm))

print(f"LightGBM - R²: {r2_lgbm:.4f}, RMSE: {rmse_lgbm:.2f}")

LightGBM - R²: 0.0912, RMSE: 590.08


# Exportando o Modelo e Encoders
Vamos salvar o modelo LightGBM treinado e os encoders necessários para uso posterior na API.

In [5]:
import joblib
import os
import json

model_dir = './model_export2'
os.makedirs(model_dir, exist_ok=True)

# Salvar modelo
model_path = os.path.join(model_dir, 'lightgbm_model.pkl')
joblib.dump(lgb_model, model_path)

# Salvar encoders
encoders_path = os.path.join(model_dir, 'encoders.pkl')
joblib.dump(encoders, encoders_path)

# Salvar informações categóricas
categorical_info = {
    'empresas': df['EMPRESA'].unique().tolist(),
    'origens': df['ORIGEM'].unique().tolist(),
    'destinos': df['DESTINO'].unique().tolist()
}

info_path = os.path.join(model_dir, 'categorical_info.json')
with open(info_path, 'w', encoding='utf-8') as f:
    json.dump(categorical_info, f, ensure_ascii=False, indent=4)

print("\nModelo e dados auxiliares exportados com sucesso!")
print(f"Modelo salvo em: {model_path}")
print(f"Encoders salvos em: {encoders_path}")
print(f"Informações categóricas salvas em: {info_path}")


Modelo e dados auxiliares exportados com sucesso!
Modelo salvo em: ./model_export2/lightgbm_model.pkl
Encoders salvos em: ./model_export2/encoders.pkl
Informações categóricas salvas em: ./model_export2/categorical_info.json


In [ ]:
import json
from pathlib import Path

# ============================================
# GERAR HISTÓRICO DE PREÇOS POR ROTA
# ============================================

# Verificar colunas esperadas
required_cols = ["ORIGEM", "DESTINO", "TARIFA"]
missing = [c for c in required_cols if c not in final_df.columns]
if missing:
    raise ValueError(f"Colunas ausentes no final_df: {missing}")

print("Gerando histórico de preços para todas rotas...")

# Agrupar por rota
grouped = final_df.groupby(["ORIGEM", "DESTINO"])["TARIFA"]

historico = []

for (orig, dest), series in grouped:
    prices = series.dropna().astype(float).tolist()
    if len(prices) == 0:
        continue
    
    historico.append({
        "origin": orig,
        "destination": dest,
        "min_price": float(min(prices)),
        "max_price": float(max(prices)),
        "prices": prices  # caso queira só min/max, pode remover
    })

# Caminho do backend
output_path = Path(".model_export2/")
output_path.parent.mkdir(parents=True, exist_ok=True)

# Salvar JSON
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(historico, f, ensure_ascii=False, indent=4)

print(f"Histórico salvo com sucesso em: {output_path}")
print(f"Total de rotas salvas: {len(historico)}")
